In [8]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from typing import Optional
import os
from dotenv import load_dotenv
load_dotenv()

True

# Routing

In [2]:
from typing import Literal
from pydantic import BaseModel, Field

class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

In [17]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

In [18]:
# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [19]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [21]:
def choose_route(result):
    match result.datasource.lower():
        case "python_docs":
            return "chain for python_docs"
        case "js_docs":
            return "chain for js_docs"
        case _:
            return "chain for golang_docs"

In [22]:
from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [23]:
full_chain.invoke({"question": question})

'chain for python_docs'

## Semantic routing

In [ ]:
math_template = """ You are a smart math teacher and explain every concept in very detail. Please answer the question: {question} and state what kind of teacher you are: """
physics_template = """ You are a smart physics teacher and explain every concept by examples in the real world. Please answer the question: {question} and state what kind of teacher you are: """

from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

def logical_router(query):
    prompt_embedding = embeddings.embed_query(query["question"])
    templates = [math_template, physics_template]
    template_embeddings = embeddings.embed_documents(templates)
    teacher_idx = cosine_similarity([prompt_embedding], template_embeddings).argmax()
    print("Using teacher: ", "Physics" if teacher_idx else "Math", "Idx: ", teacher_idx)
    return PromptTemplate.from_template(templates[teacher_idx])

routed_chain = {"question": RunnablePassthrough()} | RunnableLambda(logical_router) | ChatOpenAI() | StrOutputParser()
routed_chain.invoke("What is gravity?")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1523892441.py, line 18)

In [2]:
routed_chain.invoke("What are imaginary numbers?")

'Imaginary numbers are a mathematical concept that involves the square root of negative numbers. In the real world, imaginary numbers are often used in electrical engineering to represent currents and voltages in circuit analysis. They are also used in quantum mechanics to describe certain properties of particles.\n\nAs a physics teacher, I believe in using real-world examples to explain complex concepts. By relating imaginary numbers to practical applications in fields like electrical engineering and quantum mechanics, students can better understand the significance and relevance of this mathematical concept.'

# Query construction